In [0]:
-- CTE to calculate the total number of claims for each beneficiary
WITH claim_count_by_beneficiary AS (
    SELECT
        COUNT(claim_id) AS claim_count,
        beneficiary_key
    FROM
        gold.fact_patient_claims
    GROUP BY
        beneficiary_key
),
-- CTE to calculate claim counts and number of beneficiaries by demographic (age, gender, state)
claim_counts_by_demographic AS (
    SELECT
        COUNT(fpc.claim_id) AS claim_count,
        COUNT(DISTINCT b.beneficiary_key) AS number_of_beneficiaries,
        gold.GET_AGE(b.date_of_birth) AS age,
        b.gender,
        b.state
    FROM
        gold.dim_beneficiary AS b
    JOIN
        gold.fact_patient_claims AS fpc ON b.beneficiary_key = fpc.beneficiary_key
    GROUP BY
        b.gender,
        b.state,
        gold.GET_AGE(b.date_of_birth)
),

-- CTE to calculate the first claim start date for each beneficiary
first_claim_date_by_beneficiary AS (
    SELECT
        beneficiary_key,
        MIN(claim_start_date) AS first_claim_date
    FROM
        gold.fact_patient_claims
    GROUP BY
        beneficiary_key
)

-- Final SELECT statement to retrieve beneficiary details, total claims, average claims per demographic, and first claim start date
SELECT
    b.beneficiary_key,
    b.gender,
    b.state,
    gold.GET_AGE(b.date_of_birth) AS age,
    ccb.claim_count,
    ccd.claim_count / ccd.number_of_beneficiaries AS avg_claims_per_demographic,
    fcd.first_claim_date
FROM
    gold.dim_beneficiary AS b
JOIN
    claim_count_by_beneficiary AS ccb ON b.beneficiary_key = ccb.beneficiary_key
JOIN
    claim_counts_by_demographic AS ccd ON b.gender = ccd.gender
    AND b.state = ccd.state
    AND gold.GET_AGE(b.date_of_birth) = ccd.age
JOIN
    first_claim_date_by_beneficiary AS fcd ON b.beneficiary_key = fcd.beneficiary_key